- performans rating = (ukupan broj ratinga protivnika + 400*(pobjede-porazi))/broj partija

za podatke https://github.com/WarrierRajeev/UFC-Predictions
elo https://en.wikipedia.org/wiki/Elo_rating_system

- mozes scrapenja opisati
- inputaj si podatke
- nacrtaj neke grafove
- objasni elo
- primjeni elo neki svoj/napravljen
- objasni sto je submission sto udarac
- kritika elo sistema 
- preporuka novog
- chessboxing

In [49]:
import pandas as pd
import numpy as np
import operator

In [3]:
data = pd.read_csv("data.csv", encoding='utf-8')

In [5]:
data.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Sean O'Malley,Thomas Almeida,Mark Smith,2021-03-27,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.085938,0.789062,...,2.0,2.0,0.0,0.0,Switch,180.34,182.88,135.0,29.0,26.0
1,Jared Gooden,Abubakar Nurmagomedov,Mark Smith,2021-03-27,"Las Vegas, Nevada, USA",Blue,False,Welterweight,0.000000,0.000000,...,0.0,0.0,0.0,0.0,Orthodox,182.88,195.58,170.0,31.0,27.0
2,Alonzo Menifield,Fabio Cherant,Herb Dean,2021-03-27,"Las Vegas, Nevada, USA",Red,False,LightHeavyweight,NaN,NaN,...,0.0,2.0,0.0,0.0,Orthodox,182.88,193.04,205.0,26.0,33.0
3,Stipe Miocic,Francis Ngannou,Herb Dean,2021-03-27,"Las Vegas, Nevada, USA",Blue,True,Heavyweight,0.824707,0.000000,...,5.0,9.0,0.0,0.0,Orthodox,193.04,203.20,240.0,34.0,38.0
4,Shane Young,Omar Morales,Chris Tognoni,2021-03-27,"Las Vegas, Nevada, USA",Blue,False,Featherweight,0.250000,0.500000,...,1.0,1.0,0.0,0.0,Orthodox,172.72,182.88,145.0,35.0,27.0


In [6]:
fighter_details = pd.read_csv("fighter_details.csv", encoding='utf-8')

In [8]:
fighter_details.head()

,fighter_name,Height,Weight,Reach,Stance,DOB,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg
0,Ricardo Abreu,"5' 11""",185 lbs.,NaN,Orthodox,"Apr 27, 1984",3.79,31%,3.98,68%,2.13,42%,100%,0.7
1,Papy Abedi,"5' 11""",185 lbs.,NaN,Southpaw,"Jun 30, 1978",2.80,55%,3.15,48%,3.47,57%,50%,1.3
2,Klidson Abreu,"6' 0""",205 lbs.,"74""",Orthodox,"Dec 24, 1992",2.05,40%,2.90,55%,0.64,20%,80%,0.0
3,Shamil Abdurakhimov,"6' 3""",235 lbs.,"76""",Orthodox,"Sep 02, 1981",2.45,44%,2.45,58%,1.23,24%,47%,0.2
4,Daniel Acacio,"5' 8""",180 lbs.,NaN,Orthodox,"Dec 27, 1977",3.52,36%,2.85,62%,0.33,20%,81%,0.0


In [9]:
 preprocessed_data = pd.read_csv("preprocessed_data.csv", encoding='utf-8')

In [11]:
preprocessed_data.head()

,Winner,title_bout,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,...,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch
0,Red,False,0.085938,0.789062,0.362734,0.344766,0.00,0.159609,0.000000,0.00000,...,0,0,0,1,0,0,0,0,0,1
1,Blue,False,0.000000,0.000000,0.440000,0.180000,1.00,0.000000,0.000000,2.00000,...,0,1,0,0,0,0,1,0,0,0
2,Red,False,0.015625,0.000000,0.450000,0.427500,0.25,0.200000,0.148438,0.09375,...,0,0,0,1,0,0,1,0,0,0
3,Blue,True,0.824707,0.000000,0.488481,0.740894,0.00,0.007778,0.017578,0.00000,...,0,1,0,0,0,0,1,0,0,0
4,Blue,False,0.250000,0.500000,0.290000,0.337500,0.25,0.000000,0.000000,0.00000,...,0,1,0,0,0,0,1,0,0,0


In [16]:
total_fight_data = pd.read_csv("total_fight_data.csv", delimiter=";", encoding='utf-8')

In [20]:
total_fight_data.head()

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,...,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Sean O'Malley,Thomas Almeida,2,0,86 of 149,25 of 71,57%,35%,91 of 157,25 of 71,...,2 of 5,KO/TKO,3,3:52,3 Rnd (5-5-5),Mark Smith,"March 27, 2021","Las Vegas, Nevada, USA",Bantamweight Bout,Sean O'Malley
1,Jared Gooden,Abubakar Nurmagomedov,0,0,43 of 136,60 of 120,31%,50%,111 of 207,100 of 164,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mark Smith,"March 27, 2021","Las Vegas, Nevada, USA",Welterweight Bout,Abubakar Nurmagomedov
2,Alonzo Menifield,Fabio Cherant,0,0,3 of 3,0 of 1,100%,0%,3 of 3,0 of 1,...,0 of 0,Submission,1,1:11,3 Rnd (5-5-5),Herb Dean,"March 27, 2021","Las Vegas, Nevada, USA",Light Heavyweight Bout,Alonzo Menifield
3,Stipe Miocic,Francis Ngannou,0,2,12 of 13,36 of 56,92%,64%,12 of 13,38 of 58,...,11 of 12,KO/TKO,2,0:52,5 Rnd (5-5-5-5-5),Herb Dean,"March 27, 2021","Las Vegas, Nevada, USA",UFC Heavyweight Title Bout,Francis Ngannou
4,Shane Young,Omar Morales,0,0,46 of 105,69 of 144,43%,47%,59 of 120,74 of 150,...,1 of 1,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Chris Tognoni,"March 27, 2021","Las Vegas, Nevada, USA",Featherweight Bout,Omar Morales


In [25]:
total_fight_data.sort_values(by = ["date"], ascending = True)

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,...,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
4942,Rob Kimmons,Joe Vedepo,0,0,7 of 15,4 of 12,46%,33%,13 of 21,12 of 20,...,0 of 0,Submission,1,1:54,3 Rnd (5-5-5),Herb Dean,"April 01, 2009","Nashville, Tennessee, USA",Middleweight Bout,Rob Kimmons
4932,Ricardo Almeida,Matt Horwich,0,0,41 of 67,25 of 39,61%,64%,79 of 107,52 of 66,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Dan Miragliotta,"April 01, 2009","Nashville, Tennessee, USA",Middleweight Bout,Ricardo Almeida
4934,Martin Kampmann,Carlos Condit,0,0,26 of 61,35 of 86,42%,40%,52 of 91,71 of 124,...,8 of 9,Decision - Split,3,5:00,3 Rnd (5-5-5),Herb Dean,"April 01, 2009","Nashville, Tennessee, USA",Welterweight Bout,Martin Kampmann
4935,Cole Miller,Junie Browning,0,0,3 of 16,1 of 3,18%,33%,5 of 18,1 of 3,...,0 of 0,Submission,1,1:58,3 Rnd (5-5-5),Herb Dean,"April 01, 2009","Nashville, Tennessee, USA",Lightweight Bout,Cole Miller
4936,Tim Credeur,Nick Catone,0,0,9 of 39,19 of 37,23%,51%,10 of 40,37 of 56,...,11 of 15,Submission,2,3:45,3 Rnd (5-5-5),Mario Yamasaki,"April 01, 2009","Nashville, Tennessee, USA",Middleweight Bout,Tim Credeur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3976,Brad Pickett,Yves Jabouin,1,0,14 of 48,26 of 57,29%,45%,14 of 48,26 of 57,...,0 of 0,KO/TKO,1,3:40,3 Rnd (5-5-5),Leon Roberts,"September 29, 2012","Nottingham, England, United Kingdom",Bantamweight Bout,Brad Pickett
3975,Kyle Kingsbury,Jimi Manuwa,0,1,28 of 44,53 of 77,63%,68%,58 of 74,55 of 79,...,2 of 2,TKO - Doctor's Stoppage,2,5:00,3 Rnd (5-5-5),Leon Roberts,"September 29, 2012","Nottingham, England, United Kingdom",Light Heavyweight Bout,Jimi Manuwa
3974,Che Mills,Duane Ludwig,0,0,8 of 13,2 of 10,61%,20%,16 of 25,5 of 13,...,0 of 1,KO/TKO,1,2:28,3 Rnd (5-5-5),Herb Dean,"September 29, 2012","Nottingham, England, United Kingdom",Welterweight Bout,Che Mills
3973,Dan Hardy,Amir Sadollah,0,0,65 of 141,61 of 134,46%,45%,102 of 180,70 of 144,...,1 of 1,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Marc Goddard,"September 29, 2012","Nottingham, England, United Kingdom",Welterweight Bout,Dan Hardy


In [23]:
total_fight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6022 entries, 0 to 6021
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   R_fighter        6022 non-null   object
 1   B_fighter        6022 non-null   object
 2   R_KD             6022 non-null   int64 
 3   B_KD             6022 non-null   int64 
 4   R_SIG_STR.       6022 non-null   object
 5   B_SIG_STR.       6022 non-null   object
 6   R_SIG_STR_pct    6022 non-null   object
 7   B_SIG_STR_pct    6022 non-null   object
 8   R_TOTAL_STR.     6022 non-null   object
 9   B_TOTAL_STR.     6022 non-null   object
 10  R_TD             6022 non-null   object
 11  B_TD             6022 non-null   object
 12  R_TD_pct         6022 non-null   object
 13  B_TD_pct         6022 non-null   object
 14  R_SUB_ATT        6022 non-null   int64 
 15  B_SUB_ATT        6022 non-null   int64 
 16  R_REV            6022 non-null   int64 
 17  B_REV            6022 non-null   

In [39]:
#saznati tko ima najviše pobjeda u ufc-u
winners = np.array([row["Winner"] for index, row in total_fight_data.iterrows()])
fighter_wins = {}
for fighter in winners:
    fighter_wins[fighter]= fighter_wins[fighter] + 1 if fighter in fighter_wins else 1

In [53]:
f_w_array = [(key,int(fighter_wins[key])) for key in fighter_wins]

In [45]:
len(f_w_array)

1441

In [54]:
sorted(f_w_array, key = operator.itemgetter(1), reverse = True)

[('nan', 110),
 ('Donald Cerrone', 23),
 ('Demian Maia', 22),
 ('Jim Miller', 21),
 ('Jon Jones', 20),
 ('Georges St-Pierre', 20),
 ('Michael Bisping', 20),
 ('Dustin Poirier', 19),
 ('Rafael Dos Anjos', 19),
 ('Andrei Arlovski', 19),
 ('Diego Sanchez', 19),
 ('Max Holloway', 18),
 ('Charles Oliveira', 18),
 ('Frankie Edgar', 18),
 ('Matt Hughes', 18),
 ('Neil Magny', 17),
 ('Anderson Silva', 17),
 ('Derrick Lewis', 16),
 ('Clay Guida', 16),
 ('Francisco Trinaldo', 16),
 ('Lyoto Machida', 16),
 ('Frank Mir', 16),
 ('Gleison Tibau', 16),
 ('Randy Couture', 16),
 ('Chuck Liddell', 16),
 ('Darren Elkins', 15),
 ('Glover Teixeira', 15),
 ('Edson Barboza', 15),
 ('Matt Brown', 15),
 ('Joe Lauzon', 15),
 ('Nate Diaz', 15),
 ('Joseph Benavidez', 15),
 ('Tony Ferguson', 15),
 ('Junior Dos Santos', 15),
 ('Thiago Alves', 15),
 ('Jeremy Stephens', 15),
 ('Demetrious Johnson', 15),
 ('Vitor Belfort', 15),
 ('Ryan Bader', 15),
 ('Josh Koscheck', 15),
 ('Tito Ortiz', 15),
 ('Amanda Nunes', 14),
 ('